In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/Share/Research/speechNER/split1.zip
!unzip /content/drive/MyDrive/Share/Research/speechNER/split2.zip

Archive:  /content/drive/MyDrive/Share/Research/speechNER/split1.zip
 extracting: split1/SPEAKER0001.zip  
 extracting: split1/SPEAKER0002.zip  
 extracting: split1/SPEAKER0003.zip  
 extracting: split1/SPEAKER0004.zip  
 extracting: split1/SPEAKER0005.zip  
 extracting: split1/SPEAKER0006.zip  
 extracting: split1/SPEAKER0007.zip  
 extracting: split1/SPEAKER0008.zip  
 extracting: split1/SPEAKER0009.zip  
 extracting: split1/SPEAKER0010.zip  
 extracting: split1/SPEAKER0011.zip  
 extracting: split1/SPEAKER0012.zip  
 extracting: split1/SPEAKER0013.zip  
 extracting: split1/SPEAKER0014.zip  
 extracting: split1/SPEAKER0016.zip  
 extracting: split1/SPEAKER0017.zip  
 extracting: split1/SPEAKER0018.zip  
 extracting: split1/SPEAKER0019.zip  
 extracting: split1/SPEAKER0020.zip  
 extracting: split1/SPEAKER0021.zip  
 extracting: split1/SPEAKER0022.zip  
 extracting: split1/SPEAKER0023.zip  
 extracting: split1/SPEAKER0024.zip  
 extracting: split1/SPEAKER0025.zip  
 extracting: split1

In [ ]:
!mv split2/* split1

# Import necessary libraries and set up initial variables

In [ ]:
!pip install whisperspeech


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.6/630.6 kB 8.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylin

In [ ]:
import os
import random
import zipfile
import torch
import torch.nn.functional as F
import torchaudio
import nltk
from nltk.tokenize import sent_tokenize
import re
from whisperspeech.pipeline import Pipeline

# Define folders
speaker_folder = "split1"
temp_dir = "temp_speaker_files"
jsonl_file = "/content/drive/MyDrive/Share/Research/speechNER/Preprocessed_data_for_audio_generation.jsonl" #Final_augmented_data.jsonl"
output_dir = "/content/drive/MyDrive/Share/Research/speechNER/VC_train"
CPS = 7.5

# Create directories if they don't exist
if not os.path.exists(temp_dir):
    os.makedirs(temp_dir)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Initialize Pipeline
pipe = Pipeline(torch_compile=True)
nltk.download('punkt')

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Define helper functions

In [ ]:
CPS = 7.5

In [ ]:
def split_and_prepare_text(text, cps=CPS): #7.5):
    chunks = []
    sentences = sent_tokenize(text)
    chunk = ""
    for sentence in sentences:
        #print("bf:",sentence)
        #sentence = re.sub("(ppb)"," ppb ", sentence).strip()
        sentence = re.sub('[()]', ",", sentence)
        #sentence = re.sub(",+", ",", sentence)
        #sentence = re.sub('"+', "", sentence)
        #sentence = re.sub("/", "", sentence)
        #sentence = re.sub("\\", " ", sentence)
        #sentence = re.sub("\[", " ", sentence)
        #sentence = re.sub("\]", " ", sentence)
        #sentence = re.sub("^,", " ", sentence)
        #sentence = re.sub("_", " ", sentence)
        #sentence = re.sub("[, ]+", ",", sentence)
        sentence = sentence + "."  # Ensure each sentence ends with a period
        if len(chunk) + len(sentence) < 20*cps:
            chunk += " " + sentence
            print("this chunk:", chunk)
        else:
            if len(chunk) >0:
              chunks.append(chunk)
              print("append:",chunk,"len chunk",len(chunk))
            chunk = sentence
            print("set chunk:",chunk)
    if chunk: chunks.append(chunk)
    print("chunks:",chunks)
    return chunks

def generate_long(text, cps=CPS, overlap=100, output=None, speaker=None):
    global atoks, stoks
    chunks = split_and_prepare_text(text)
    speaker = pipe.default_speaker if speaker is None else speaker
    r = []
    old_stoks = None
    old_atoks = None
    for chunk in chunks:
        print(chunk)
        stoks = pipe.t2s.generate(chunk, cps=cps, show_progress_bar=False)[0]
        stoks = stoks[stoks != 512]
        if old_stoks is not None:
            assert(len(stoks) < 750-overlap)
            stoks = torch.cat([old_stoks[-overlap:], stoks])
            atoks_prompt = old_atoks[:,:,-overlap*3:]
        else:
            atoks_prompt = None
        atoks = pipe.s2a.generate(stoks, atoks_prompt=atoks_prompt, speakers=speaker.unsqueeze(0), show_progress_bar=False)
        if atoks_prompt is not None: atoks = atoks[:,:,overlap*3+1:]
        r.append(atoks)
        old_stoks = stoks
        old_atoks = atoks
        pipe.vocoder.decode_to_notebook(atoks)
    audios = []
    for i,atoks in enumerate(r):
        if i != 0: audios.append(torch.zeros((1, int(24000*0.5)), dtype=atoks.dtype, device=atoks.device))
        audios.append(pipe.vocoder.decode(atoks))
    if output:
        torchaudio.save(output, torch.cat(audios, -1).cpu(), 24000)
        from IPython.display import display, HTML, Audio
        display(HTML(f'Listen to {output}'))

# Function to collect and extract WAV files

In [ ]:
def collect_all_wav_files():
    all_wav_files = []
    zip_files = [f for f in os.listdir(speaker_folder) if f.endswith('.zip')]

    print(f"Found {len(zip_files)} zip files in {speaker_folder}")

    for zip_filename in zip_files:
        zip_path = os.path.join(speaker_folder, zip_filename)
        print(f"\nProcessing zip file: {zip_filename}")
        try:
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                speaker_name = zip_filename.replace('.zip', '')
                all_files = zip_ref.namelist()
                print(f"Total files in zip: {len(all_files)}")

                wav_files = [
                    f for f in all_files
                    if f.upper().endswith('.WAV') and
                    f.startswith(f"{speaker_name}/SESSION")
                ]
                print(f"WAV files found: {len(wav_files)}")
                if wav_files:
                    print("First 5 WAV files:")
                    for wav in wav_files[:5]:
                        print(f"  {wav}")
                all_wav_files.extend([(zip_filename, wav_file) for wav_file in wav_files])
        except zipfile.BadZipFile:
            print(f"Error: {zip_filename} is not a valid zip file.")
        except Exception as e:
            print(f"An error occurred while processing {zip_filename}: {str(e)}")

    print(f"\nTotal WAV files found across all zips: {len(all_wav_files)}")
    return all_wav_files

def extract_all_wav_files():
    all_wav_files = collect_all_wav_files()
    extracted_files = []

    if not all_wav_files:
        print("No .wav files found in any zip file.")
        return []

    for zip_filename, wav_file in all_wav_files:
        zip_path = os.path.join(speaker_folder, zip_filename)
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extract(wav_file, temp_dir)

        extracted_path = os.path.join(temp_dir, wav_file)
        if os.path.exists(extracted_path):
            extracted_files.append(extracted_path)
            print(f"Extracted: {extracted_path}")
        else:
            print(f"Failed to extract: {wav_file}")

    print(f"\nTotal .wav files extracted: {len(extracted_files)}")
    return extracted_files

def get_random_speaker_file(extracted_files):
    if not extracted_files:
        print("No extracted .wav files available.")
        return None

    random_wav = random.choice(extracted_files)
    print(f"Selected .wav file: {random_wav}")
    return random_wav

# Prevent termination

The audio extraction and voice clone will take long time depending on the GPU allocated to this notebook.
You should make sure that the process doesn't stop due to inactivity. A simple hack to prevent this is to paste the following code into the console of this tab (right mouse click -> inspect -> Console tab and insert code).

```javascript
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton,60000);
```

# Main processing

In [ ]:
# Extract all .wav files
extracted_files = extract_all_wav_files()

Streaming output truncated to the last 5000 lines.
Extracted: temp_speaker_files/SPEAKER0049/SESSION0/000490157.WAV
Extracted: temp_speaker_files/SPEAKER0049/SESSION0/000490158.WAV
Extracted: temp_speaker_files/SPEAKER0049/SESSION0/000490159.WAV
Extracted: temp_speaker_files/SPEAKER0049/SESSION0/000490160.WAV
Extracted: temp_speaker_files/SPEAKER0049/SESSION0/000490161.WAV
Extracted: temp_speaker_files/SPEAKER0049/SESSION0/000490162.WAV
Extracted: temp_speaker_files/SPEAKER0049/SESSION0/000490163.WAV
Extracted: temp_speaker_files/SPEAKER0049/SESSION0/000490164.WAV
Extracted: temp_speaker_files/SPEAKER0049/SESSION0/000490165.WAV
Extracted: temp_speaker_files/SPEAKER0049/SESSION0/000490166.WAV
Extracted: temp_speaker_files/SPEAKER0049/SESSION0/000490167.WAV
Extracted: temp_speaker_files/SPEAKER0049/SESSION0/000490168.WAV
Extracted: temp_speaker_files/SPEAKER0049/SESSION0/000490169.WAV
Extracted: temp_speaker_files/SPEAKER0049/SESSION0/000490170.WAV
Extracted: temp_speaker_files/SPEAKER00

In [ ]:
!rm -fr {output_dir}
!mkdir {output_dir}

In [ ]:
# Read the JSONL file
import json
#jsonl_file = "/content/drive/MyDrive/Share/Research/speechNER/Final_augmented_data.jsonl"
#output_dir = "/content/drive/MyDrive/Share/Research/speechNER/VC_train"
texts = []

with open(jsonl_file, 'r') as file:
    for line in file:
        data_point = json.loads(line)
        texts.append(data_point['text'])

# Process each text
output_speaker_map = {}
#f = open(f"{output_dir}/output_speaker_map.txt", 'w')
pattern = '[\.\/]'
#for i in range(0, len(texts)):
#for i in range(469, len(texts)):
for i in range(491, len(texts)):
    text = texts[i]
#for i, text in enumerate(texts):
    #sample_text = split_and_prepare_text(text)
    #print(sample_text)
    speaker_file = get_random_speaker_file(extracted_files)

    if speaker_file:
        speaker = pipe.extract_spk_emb(speaker_file)
        sp=re.split(pattern,speaker_file)
        output_file = f"{output_dir}/train{i+1}_{sp[1]}_{sp[3]}.wav"
        #print(output_file)
        #f.write(f"{output_file}: {speaker_file}\n")
        generate_long(text, output=output_file, speaker=speaker)
        #output_speaker_map[output_file] = speaker_file
        print(f"Processed text {i+1} with speaker file: {speaker_file}")
        #break
    else:
        print(f"Skipped processing text {i+1} due to speaker file selection failure")

print("Processing complete.")

# Save the mapping to a text file
#with open(f"{output_dir}/output_speaker_map.txt", 'w') as f:
#    for output_file, speaker_file in output_speaker_map.items():
#        f.write(f"{output_file}: {speaker_file}\n")

print("Speaker mapping saved.")

Output hidden; open in https://colab.research.google.com to view.